In [1]:
!pip install --upgrade pip
!pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

!pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 102.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 69.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requir

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [4]:
huggingface_dataset_name = "databricks/databricks-dolly-15k"

train_ds = load_dataset(huggingface_dataset_name, split='train[:80%]')
test_ds = load_dataset(huggingface_dataset_name, split='train[:20%]')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


In [5]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [7]:
train_ds

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 12009
})

In [8]:
index = 200

question = train_ds[index]['instruction']
answer = train_ds[index]['response']

prompt = f"""
give a response for the following conversation.

{question}

response:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'ANSWER FROM THE DATASET:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

give a response for the following conversation.

What is wine?

response:

---------------------------------------------------------------------------------------------------
ANSWER FROM THE DATASET:
Wine is an alcoholic drink typically made from fermented grapes.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Wine (disambiguation) Wine is a wine that is made from grapes, such as grapes, grapes, or other plants.


In [9]:
def tokenize_function(example):
    start_prompt = 'Give a response for  the following question.\n\n'
    end_prompt = '\n\response: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["instruction"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["response"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 2 diff splits: train, test.
# The tokenize_function code is handling all data across all splits in batches.
train_data = train_ds.map(tokenize_function, batched=True)
train_data = train_data.remove_columns(['instruction', 'context', 'response', 'category',])
test_data = test_ds.map(tokenize_function, batched=True)
test_data = test_data.remove_columns(['instruction', 'context', 'response', 'category',])

Map:   0%|          | 0/12009 [00:00<?, ? examples/s]

Map:   0%|          | 0/3002 [00:00<?, ? examples/s]

In [10]:
print(f"Shapes of the datasets:")
print(f"Training: {train_data.shape}")
print(train_data)

Shapes of the datasets:
Training: (12009, 2)
Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 12009
})


In [11]:
print(f"Shapes of the datasets:")
print(f"Training: {test_data.shape}")
print(test_data)

Shapes of the datasets:
Training: (3002, 2)
Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 3002
})


In [15]:
output_dir = f'Flan-t5-qa-model'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    max_steps=1,
    logging_steps=1,
    push_to_hub=True
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer
)

Cloning https://huggingface.co/kla-20/Flan-t5-qa-model into local empty directory.


In [16]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,43.250000


TrainOutput(global_step=1, training_loss=43.25, metrics={'train_runtime': 5.9124, 'train_samples_per_second': 1.353, 'train_steps_per_second': 0.169, 'total_flos': 5478058819584.0, 'train_loss': 43.25, 'epoch': 0.0})

# Saving model

In [22]:
trainer.save_model('Flan-t5-qa-model')

Upload file pytorch_model.bin:   0%|          | 1.00/472M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.50k [00:00<?, ?B/s]

Upload file runs/Sep22_04-09-56_91bc47a3736b/events.out.tfevents.1695355806.91bc47a3736b.295.0:   0%|         …

Upload file runs/Sep22_04-09-56_91bc47a3736b/1695355806.295214/events.out.tfevents.1695355806.91bc47a3736b.295…

To https://huggingface.co/kla-20/Flan-t5-qa-model
   b043720..83f5496  main -> main

   b043720..83f5496  main -> main

To https://huggingface.co/kla-20/Flan-t5-qa-model
   83f5496..8c4deb2  main -> main

   83f5496..8c4deb2  main -> main



# Human Evaluation

In [17]:
train_data

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 12009
})

In [23]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./Flan-t5-qa-model", torch_dtype=torch.bfloat16)
instruct_model.to("cuda")

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [25]:
index = 200
instruction = test_ds[index]['instruction']
response = test_ds[index]['response']


prompt = f"""
Answer the following question.

{instruction}

Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
input_ids=input_ids.to("cuda")
original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN Answer:\n{response}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN Answer:
Wine is an alcoholic drink typically made from fermented grapes.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
a alcoholic drink
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
wine


# Rouge Metric

In [26]:
rouge = evaluate.load('rouge')

In [27]:
dialogues = test_ds[0:10]['instruction']
human_baseline_responses = test_ds[0:10]['response']

original_model_responses = []
instruct_model_responses = []

for _, dialogue in enumerate(dialogues):
    prompt = f"""
Answer the following question.

{instruction}

Answer: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids=input_ids.to("cuda")
    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_responses.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_responses.append(instruct_model_text_output)

zipped_responses = list(zip(human_baseline_responses, original_model_responses, instruct_model_responses))

df = pd.DataFrame(zipped_responses, columns = ['human_baseline_responses', 'original_model_responses', 'instruct_model_responses'])
df

,human_baseline_responses,original_model_responses,instruct_model_responses
0,Virgin Australia commenced services on 31 Augu...,wine,wine
1,Tope,a syringe,wine
2,Camels use the fat in their humps to keep them...,a mash,wine
3,The name of the third daughter is Alice,a type of fermented,wine
4,"Tomoaki Komorida was born on July 10,1981.",wine,wine
5,No. \nStalemate is a drawn position. It doesn'...,scotland wine,wine
6,Lollapalooze is an annual musical festival hel...,alcoholic,wine
7,John D Rockerfeller,wine,wine
8,We are always engaged one phone which is not g...,a wine,wine
9,John Moses Browning is one of the most well-kn...,a liquid,wine


# Evaluating using Rouge Metric

In [28]:
original_model_results = rouge.compute(
    predictions=original_model_responses,
    references=human_baseline_responses[0:len(original_model_responses)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_responses,
    references=human_baseline_responses[0:len(instruct_model_responses)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.016666666666666666, 'rouge2': 0.0, 'rougeL': 0.016666666666666666, 'rougeLsum': 0.016666666666666666}
INSTRUCT MODEL:
{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


# Saving into csv

In [30]:
results = df.copy()
# saving dataset into results

human_baseline_responses = results['human_baseline_responses'].values
original_model_responses = results['original_model_responses'].values
instruct_model_responses = results['instruct_model_responses'].values

original_model_results = rouge.compute(
    predictions=original_model_responses,
    references=human_baseline_responses[0:len(original_model_responses)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_responses,
    references=human_baseline_responses[0:len(instruct_model_responses)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.016666666666666666, 'rouge2': 0.0, 'rougeL': 0.016666666666666666, 'rougeLsum': 0.016666666666666666}
INSTRUCT MODEL:
{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


# Comparing results

In [31]:
print("Absolute percentage improvement of INSTRUCT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of INSTRUCT MODEL over ORIGINAL MODEL
rouge1: -1.67%
rouge2: 0.00%
rougeL: -1.67%
rougeLsum: -1.67%


# PEFT

In [12]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

Add LoRA adapter layers/parameters to the original LLM to be trained.

In [13]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%


## Train PEFT Adapter
Define training arguments and create Trainer instance.

In [14]:
output_dir = f'Peft-Flan-t5-qa-model'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1,
    push_to_hub=True
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer
)

Cloning https://huggingface.co/kla-20/Peft-Flan-t5-qa-model into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.00k/486M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.50k/3.50k [00:00<?, ?B/s]

Download file runs/Sep22_05-48-35_35bc48e2b724/1695361720.2507522/events.out.tfevents.1695361720.35bc48e2b724.…

Clean file training_args.bin:  29%|##8       | 1.00k/3.50k [00:00<?, ?B/s]

Clean file runs/Sep22_05-48-35_35bc48e2b724/1695361720.2507522/events.out.tfevents.1695361720.35bc48e2b724.342…

Download file runs/Sep22_05-48-35_35bc48e2b724/events.out.tfevents.1695361720.35bc48e2b724.342.0: 100%|#######…

Clean file runs/Sep22_05-48-35_35bc48e2b724/events.out.tfevents.1695361720.35bc48e2b724.342.0:  24%|##4       …

Clean file pytorch_model.bin:   0%|          | 1.00k/486M [00:00<?, ?B/s]

In [15]:
peft_trainer.train()

peft_model_path="./peft-Flan-T5-qa"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,45.000000


('./peft-Flan-T5-qa/tokenizer_config.json',
 './peft-Flan-T5-qa/special_tokens_map.json',
 './peft-Flan-T5-qa/tokenizer.json')

In [18]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       '/content/peft-Flan-T5-qa',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

In [19]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 0
all model parameters: 251116800
percentage of trainable model parameters: 0.00%


In [27]:
peft_model.to("cuda")

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear(
                    in_features=768, out_features=768, bias=False
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=32, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedding_B):

## The number of trainable parameters will be 0 due to is_trainable=False setting:

# Evaluate the Model Qualitatively (Human Evaluation)

In [20]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("kla-20/Flan-t5-qa-model", torch_dtype=torch.bfloat16)
instruct_model.to("cuda")

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [23]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [25]:
original_model.to("cuda")

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [28]:
index = 200

dialogues = test_ds[index]['instruction']
human_baseline_responses = test_ds[index]['response']


original_model_responses = []
instruct_model_responses = []

for _, dialogue in enumerate(dialogues):
    prompt = f"""
Answer the following question.

{dialogues}

Answer: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids=input_ids.to("cuda")

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{human_baseline_responses}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN ANSWER:
Wine is an alcoholic drink typically made from fermented grapes.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
wine
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
wine
---------------------------------------------------------------------------------------------------
PEFT MODEL: wine


## Evaluate the Model Quantitatively (with ROUGE Metric)
Perform inferences for the sample of the test dataset (only 10 dialogues and summaries to save time)

In [35]:
original_model.to("cuda")

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [38]:
instruction = test_ds[0:10]['instruction']
human_baseline_responses = test_ds[0:10]['response']

original_model_responses = []
instruct_model_responses = []
peft_model_responses = []

for idx, dialogue in enumerate(instruction):
    prompt = f"""
Answer the following Question.

{dialogue}

Summary: """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids=input_ids.to("cuda")
    human_baseline_text_output = human_baseline_responses[idx]

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_responses.append(original_model_text_output)
    instruct_model_responses.append(instruct_model_text_output)
    peft_model_responses.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_responses, original_model_responses, instruct_model_responses, peft_model_responses))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_responses', 'original_model_responses', 'instruct_model_responses', 'peft_model_responses'])
df

,human_baseline_responses,original_model_responses,instruct_model_responses,peft_model_responses
0,Virgin Australia commenced services on 31 Augu...,1 January 2001,1 January 2001,1 January 2001
1,Tope,...,...,...
2,Camels use the fat in their humps to keep them...,They are omnivorous.,They are omnivorous.,They are omnivorous.
3,The name of the third daughter is Alice,annie,annie,annie
4,"Tomoaki Komorida was born on July 10,1981.",october 19th 1939,october 19th 1939,"october 19th, 1939"
5,No. \nStalemate is a drawn position. It doesn'...,1).,1).,1).
6,Lollapalooze is an annual musical festival hel...,san francisco,san francisco,san francisco
7,John D Rockerfeller,nato,nato,nato
8,We are always engaged one phone which is not g...,Mobile phones are a major threat to human health.,Mobile phones are a major threat to human health.,Mobile phones are a major threat to human health.
9,John Moses Browning is one of the most well-kn...,john m. browning,john m. browning,john m. browning


In [39]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_responses,
    references=human_baseline_responses[0:len(original_model_responses)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_responses,
    references=human_baseline_responses[0:len(instruct_model_responses)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_responses,
    references=human_baseline_responses[0:len(peft_model_responses)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.024278438030560272, 'rouge2': 0.0, 'rougeL': 0.01375212224108659, 'rougeLsum': 0.01375212224108659}
INSTRUCT MODEL:
{'rouge1': 0.024278438030560272, 'rouge2': 0.0, 'rougeL': 0.01375212224108659, 'rougeLsum': 0.01375212224108659}
PEFT MODEL:
{'rouge1': 0.024278438030560272, 'rouge2': 0.0, 'rougeL': 0.01375212224108659, 'rougeLsum': 0.01375212224108659}


The results show less of an improvement over full fine-tuning, but the benefits of PEFT typically outweigh the slightly-lower performance metrics.

Calculate the improvement of PEFT over the original model:

In [40]:
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 0.00%
rouge2: 0.00%
rougeL: 0.00%
rougeLsum: 0.00%


Now calculate the improvement of PEFT over a full fine-tuned model:

In [41]:
print("Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(instruct_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL
rouge1: 0.00%
rouge2: 0.00%
rougeL: 0.00%
rougeLsum: 0.00%
